# Third party Data Import - PlanetScope, COVID 19

This Jupyter Notebook demonstrates how to import **PlanetScope** data into Sentinel Hub and then access it.

We will use **Simple search** and **Order products**[1].

 
Useful links:  
    [1] TPDI API documentation: https://docs.sentinel-hub.com/api/latest/#/DATA_IMPORT_API/README  
    [2] TPDI API reference (Swagger docs): https://docs.sentinel-hub.com/api/latest/reference/#tag/dataimport_planet  
    [3] About PlanetScope data: https://docs.sentinel-hub.com/api/latest/#/data/PlanetScope

## Imports and credentials

If you are running the JN localy fill in **config.py** file with your credentials. 

If you are running the JN on EDC, you will need a SH subscription and SH credentials will be automatically available. 

In [1]:
from edc import setup_environment_variables
setup_environment_variables()

API credentials have automatically been injected for your active subscriptions.  
The following environment variables are now available:
* `SH_INSTANCE_ID`
* `SH_CLIENT_ID`
* `SH_CLIENT_SECRET`
* `SH_CLIENT_NAME`


In [2]:
# Read SH credentials from environment variables
import os

client_id = os.environ['SH_CLIENT_ID']
client_secret = os.environ['SH_CLIENT_SECRET']

In [3]:
import requests 
import json
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
import os
import datetime
import dateutil.parser

# Create a session
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)


# Get token for the session
token = oauth.fetch_token(token_url='https://services.sentinel-hub.com/oauth/token',
                          client_id=client_id, client_secret=client_secret)

In [4]:
#Get my quota

url = f"https://services.sentinel-hub.com/dataimport/v1/quotas"

r = oauth.get(url=url)
r.json()

{'data': [{'id': 'd2c3ee95-f6f3-4035-b14c-529a8a4c753d',
   'datasetId': 'PLANET_SCOPE',
   'quotaSqkm': 50.0,
   'quotaUsed': 0.0}]}

## Search

In [5]:
url = "https://services.sentinel-hub.com/dataimport/v1/search"

aoi = {'type': 'Polygon',
  'coordinates': [[[12.560804463221928, 41.885177999999996],
    [12.585804463221926, 41.885177999999996],
    [12.585804463221926, 41.910178],
    [12.560804463221928, 41.910178],
    [12.560804463221928, 41.885177999999996]]]}

aoi = {"type": "Polygon", "coordinates": [[[12.567054463221927, 41.891428], [12.579554463221926, 41.891428], [12.579554463221926, 41.903928], [12.567054463221927, 41.903928], [12.567054463221927, 41.891428]]]}

query = {
    "provider": "PLANET",
    "planetApiKey": PLANET_API_KEY,
    "bounds": {
        "geometry": aoi
    },
    "data": [
        {
            "itemType": "PSScene4Band",
            "dataFilter": {
                "timeRange": {
                    "from": "2020-04-15T00:00:00.000Z",
                    "to": "2020-04-17T00:00:00.000Z"
                },
                "maxCloudCoverage": 30
            }
        }
    ]
}

response = oauth.post(url, json=query)
response.raise_for_status()

results = response.json()

In [76]:
search_results = [(feature["id"], feature["properties"]['acquired'], \
                   feature["properties"]['cloud_cover']) \
                   for feature in results["features"]]
print("Product id \t acquisition date&time \t cloudcover  ")
for item in search_results:
    print(item[0],   item[1],   item[2])

Product id 	 acquisition date&time 	 cloudcover  
20200416_093843_1003 2020-04-16T09:38:43.436748Z 0
20200415_100513_35_1058 2020-04-15T10:05:13.355052Z 0.03
20200415_091216_83_2257 2020-04-15T09:12:16.833103Z 0
20200415_100515_37_1058 2020-04-15T10:05:15.370415Z 0.03


In [77]:
# Additional filtering of search results by cloud coverage
item_ids = [feature['id'] for feature in results["features"] if feature["properties"]['cloud_cover'] == 0]
item_ids

['20200416_093843_1003', '20200415_091216_83_2257']

## Order data import

In [78]:
url = "https://services.sentinel-hub.com/dataimport/v1/orders"

query['data'][0]["productBundle"] = "analytic"

name = "anja test"
payload = {
    "name": name,
    "input": query
}

response = oauth.post(url, json=payload)
response.raise_for_status()

order = response.json()

In [79]:
order_id = order['id']
area = order['sqkm']
print(f"Order {name}, id: {order_id}, area: {area} km^2")

Order anja test, id: 109af2d3-b72e-4304-ba8c-a0d5d8f8dd96, area: 4.968239307460755 km^2


## Confirm the order

In [84]:
url = f"https://services.sentinel-hub.com/dataimport/v1/orders/{order_id}/confirm"

response = oauth.post(url)
response.raise_for_status()

In [87]:
response.json()

{'id': '109af2d3-b72e-4304-ba8c-a0d5d8f8dd96',
 'userId': '42f086c7-d327-4e93-ac15-48008abf24b2',
 'created': '2020-04-24T10:52:27.858Z',
 'name': 'anja test',
 'collectionId': '5a4650d9-d0c1-4ec1-b32f-864e081380d2',
 'provider': 'PLANET',
 'input': {'bounds': {'geometry': {'type': 'Polygon',
    'coordinates': [[[12.567054463221927, 41.891428],
      [12.579554463221926, 41.891428],
      [12.579554463221926, 41.903928],
      [12.567054463221927, 41.903928],
      [12.567054463221927, 41.891428]]]},
   'properties': {}},
  'data': [{'dataFilter': {'timeRange': {'from': '2020-04-15T00:00:00Z',
      'to': '2020-04-17T00:00:00Z'},
     'maxCloudCoverage': 30.0},
    'itemType': 'PSScene4Band',
    'productBundle': 'analytic'}],
  'planetApiKey': '76f56ada21b8419890043d7d870c67bf',
  'provider': 'PLANET'},
 'sqkm': 4.968239307460755,
 'status': 'CONFIRMED'}

# Check Status

In [90]:
order_id = order_id

url = f"https://services.sentinel-hub.com/dataimport/v1/orders/{order_id}"

response = oauth.get(url)
response.raise_for_status()

order_status = response.json()
order_status

{'id': '109af2d3-b72e-4304-ba8c-a0d5d8f8dd96',
 'userId': '42f086c7-d327-4e93-ac15-48008abf24b2',
 'created': '2020-04-24T10:52:27.858Z',
 'name': 'anja test',
 'collectionId': '5a4650d9-d0c1-4ec1-b32f-864e081380d2',
 'provider': 'PLANET',
 'input': {'bounds': {'geometry': {'type': 'Polygon',
    'coordinates': [[[12.567054463221927, 41.891428],
      [12.579554463221926, 41.891428],
      [12.579554463221926, 41.903928],
      [12.567054463221927, 41.903928],
      [12.567054463221927, 41.891428]]]},
   'properties': {}},
  'data': [{'dataFilter': {'timeRange': {'from': '2020-04-15T00:00:00Z',
      'to': '2020-04-17T00:00:00Z'},
     'maxCloudCoverage': 30.0},
    'itemType': 'PSScene4Band',
    'productBundle': 'analytic'}],
  'planetApiKey': '76f56ada21b8419890043d7d870c67bf',
  'provider': 'PLANET'},
 'sqkm': 4.968239307460755,
 'status': 'CONFIRMED'}

## List all orders

In [91]:
url = "https://services.sentinel-hub.com/dataimport/v1/orders"

response = oauth.get(url)
response.raise_for_status()

response.json()

{'data': [{'id': '0122458c-4f49-40af-80c5-82adf504745a',
   'userId': '42f086c7-d327-4e93-ac15-48008abf24b2',
   'created': '2020-04-09T21:29:38.854Z',
   'name': 'South Budapest Industry estate, 2020',
   'collectionId': '3c4daecf-09f3-451c-8c3c-90e356cbd673',
   'provider': 'AIRBUS',
   'input': {'bounds': {'geometry': {'type': 'Polygon',
      'coordinates': [[[19.083411598048095, 47.32569590870299],
        [19.160900110588322, 47.32569590870299],
        [19.160900110588322, 47.38095740946729],
        [19.083411598048095, 47.38095740946729],
        [19.083411598048095, 47.32569590870299]]]},
     'properties': {}},
    'data': [{'constellation': 'PHR',
      'products': [{'id': 'f1ddfa74-68f8-4bcd-9cad-e57bd32d0257',
        'crsCode': 'urn:ogc:def:crs:EPSG::32634'}]}],
    'provider': 'AIRBUS'},
   'sqkm': 35.96,
   'status': 'DONE'},
  {'id': '01d61988-6aa2-4eeb-a293-890cb24372f2',
   'userId': '42f086c7-d327-4e93-ac15-48008abf24b2',
   'created': '2020-04-02T13:44:07.174Z',
 